In [76]:
from splinter import Browser
from bs4 import BeautifulSoup as soup
import re
import pandas as pd
import matplotlib.pyplot as plt
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from unidecode import unidecode
import emoji
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementNotInteractableException
from googletrans import Translator
from langdetect import detect
from collections import Counter
import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [2]:
# Set up search parameters
min_price = 1000
max_price = 30000
days_listed = 7
min_mileage = 50000
max_mileage = 200000
min_year = 2000
max_year = 2020
transmission = "automatic"
query = "Honda Civic"
#"102170323157613" = Moncton NB, "114723638540069" = Timmins ON, "111551465530472" = Thunder Bay ON
cities = ["102170323157613", "montreal", "toronto", "114723638540069", "111551465530472", "winnipeg", "regina", "calgary", "vancouver"]

In [3]:
browser = webdriver.Chrome()

# Create a list to store htmls
soups = []

for i, city in enumerate(cities):
    # Set up base url
    base_url = f'https://www.facebook.com/marketplace/{city}/search?'
    #Set up full url
    url = f"{base_url}minPrice={min_price}&maxPrice={max_price}&daysSinceListed={days_listed}&maxMileage={max_mileage}&maxYear={max_year}&minMileage={min_mileage}&minYear={min_year}&transmissionType={transmission}&query={query}&exact=false"

    # Visit the website
    browser.get(url)

    # Find the div element by its CSS selector
    wait = WebDriverWait(browser, 10)
    div_element = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'div.x1i10hfl.x1qjc9v5.xjbqb8w.xjqpnuy.xa49m3k.xqeqjp1.x2hbi6w.x13fuv20.xu3j5b3.x1q0q8m5.x26u7qi.x972fbf.xcfux6l.x1qhh985.xm0m39n.x9f619.x1ypdohk.xdl72j9.x2lah0s.xe8uvvx.x11i5rnm.xat24cr.x1mh8g0r.x2lwn1j.xeuugli.xexx8yu.x4uap5.x18d9i69.xkhd6sd.x1n2onr6.x16tdsg8.x1hl2dhg.xggy1nq.x1ja2u2z.x1t137rt.x1o1ewxj.x3x9cwd.x1e5q0jg.x13rtm0m.x1q0g3np.x87ps6o.x1lku1pv.x78zum5.x1a2a7pz.x1xmf6yo[tabindex="0"]')))

    # Click on the div element
    div_element.click()

    # Wait for the label element to be clickable
    wait = WebDriverWait(browser, 5)
    
    # Account for variation in label element containing search radius (km)
    try:
        label_element = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'label[for=":rj:"]')))
    except:
        try:
            label_element = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'label[for=":re:"]')))
        except:
            try:
                label_element = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'label[for=":rd:"]')))
            except:
                try:
                    label_element = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'label[for=":rk:"]')))
                except:
                    label_element = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'label[for=":rm:"]')))
                

    # Click on the label element
    label_element.click()

    # Press the "Down" key 11 times
    for _ in range(12):
        label_element.send_keys(Keys.DOWN)

    # Press Enter
    label_element.send_keys(Keys.ENTER)

    # Wait for the "Apply" button to be clickable
    wait = WebDriverWait(browser, 10)
    apply_div = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'div[aria-label="Apply"]')))

    # Click on the div element
    apply_div.click()
    
    # Scroll down to load more results
    scroll_count = 3
    scroll_delay = 1

    for _ in range(scroll_count):
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_delay)
    
    # Parse the HTML
    html = browser.page_source
    
    # Create a BeautifulSoup object from the scraped HTML
    soups.append(soup(html, 'html.parser'))
    
    # Check if HTML was scraped correctly
    print(f'finished {city}')
    print('--------------------------------')

finished 102170323157613
--------------------------------
finished montreal
--------------------------------
finished toronto
--------------------------------
finished 114723638540069
--------------------------------
finished 111551465530472
--------------------------------
finished winnipeg
--------------------------------
finished regina
--------------------------------
finished calgary
--------------------------------
finished vancouver
--------------------------------


In [4]:
# Close the browser
browser.quit()

In [5]:
# Create empty lists to store the extracted information
titles_list = []
prices_list = []
mileage_list = []
urls_list = []

# Iterate over the soups
for soup in soups:
    # Extract title information
    titles_div = soup.find_all('span', class_="x1lliihq x6ikm8r x10wlt62 x1n2onr6")
    titles_list.extend([title.text.strip() for title in titles_div])
    
    # Extract price information
    prices_div = soup.find_all('span', class_="x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x xudqn12 x676frb x1lkfr7t x1lbecb7 x1s688f xzsf02u")
    prices_list.extend([price.text.strip() for price in prices_div])
    
    # Extract mileage information
    mileage_div = soup.find_all('span', class_="x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x x4zkp8e x3x7a5m x1nxh6w3 x1sibtaa xo1l8bm xi81zsa")
    mileage_list.extend([mileage.text.strip() for mileage in mileage_div])
    
    # Extract URL information
    urls_div = soup.find_all('a', class_="x1i10hfl xjbqb8w x6umtig x1b1mbwd xaqea5y xav7gou x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1a2a7pz x1heor9g x1lku1pv")
    urls_list.extend([url.get('href') for url in urls_div])

In [6]:
# Set patterns for mileage and location
mileage_pattern_km = r'(\d+)K km'
mileage_pattern_miles = r'(\d+)K miles'
location_pattern = r'\b[\w\s-]+(?:, \w{2})\b'

# Extract mileage, city and province according to pattern and transform values into integers
mileage_clean = []
city_data = []
province_data = []

# Iterate through mileage list to separate location from mileage
for item in mileage_list:
    match_mileage_km = re.search(mileage_pattern_km, item)
    match_mileage_miles = re.search(mileage_pattern_miles, item)
    match_location = re.search(location_pattern, item)

    # Check for mileage pattern using regex
    if match_mileage_km or match_mileage_miles:
        if match_mileage_km:
            mileage_clean.append(int(match_mileage_km.group(1)) * 1000)
        else:
            mileage_clean.append(int(match_mileage_miles.group(1)) * 1000)
            
    # Check for location pattern using regex     
    elif match_location:
        location = match_location.group(0)
        city, province = map(str.strip, location.split(','))
        city_data.append(city)
        province_data.append(province)
        
    elif item == "":
        mileage_clean.append(0)

# Check for equal length on all lists        
print(len(city_data), len(province_data), len(mileage_clean))
city_data

550 550 550


['Dieppe',
 'Riverview',
 'Moncton',
 'Charlottetown',
 'Halifax',
 'Saint John',
 'Neguac',
 'Halifax',
 'Québec',
 'Halifax',
 'Moncton Parish',
 'Halifax',
 'Lac-Baker Parish',
 'Fredericton',
 'Fredericton',
 'Halifax',
 'Fredericton',
 'Paspébiac',
 'Bathurst',
 'Bathurst',
 'Summerside',
 'Lunenburg Municipal District',
 'Warwick',
 'Québec',
 'Halifax',
 'Lévis',
 'Fredericton',
 'Québec',
 'Fredericton',
 'Sherbrooke',
 'Moncton',
 'Sherbrooke',
 'Halifax',
 'Halifax',
 'Bathurst',
 'Halifax',
 'Bathurst',
 'Rothesay',
 'Saint John',
 'Charlottetown',
 'East Hants',
 'Murdochville',
 'St-Georges',
 'Bathurst',
 'Edmundston',
 'Sept-Îles',
 'Halifax',
 'Halifax',
 'Québec',
 'Lévis',
 'Halifax',
 'Halifax',
 'Québec',
 'Cape Breton',
 'Lévis',
 'Thetford Mines',
 'Val-Joli',
 'Cornwall',
 'Halifax',
 'St-Georges',
 'Blackville',
 'Québec',
 'Ste-Marie',
 'New Richmond',
 'Sherbrooke',
 'Québec',
 'New Richmond',
 'Québec',
 'Lévis',
 'Lévis',
 'Lévis',
 'Québec',
 'Montréal',
 '

In [7]:
# Add all values to a list of dictionaries
vehicles_list = []

for i, item in enumerate(titles_list):
    cars_dict = {}
    
    title_split = titles_list[i].split()
    
    cars_dict["City"] = city_data[i]
    cars_dict["Province"] = province_data[i]
    cars_dict["Year"] = int(title_split[0])
    
    # Use a try-except block to handle model information
    try:
        cars_dict["Make"] = title_split[1]
        cars_dict["Model"] = title_split[2]
    except IndexError:
        # If model information is not present or in a different format, set default values or skip the entry
        cars_dict["Make"] = "Unknown Make"
        cars_dict["Model"] = "Unknown Model"
    
    cars_dict["Price"] = int(re.sub(r'[^\d.]', '', prices_list[i]))
    cars_dict["Mileage"] = mileage_clean[i]
    cars_dict["URL"] = urls_list[i]
    vehicles_list.append(cars_dict)
    
print(vehicles_list)

[{'City': 'Dieppe', 'Province': 'NB', 'Year': 2015, 'Make': 'Honda', 'Model': 'civic', 'Price': 14000, 'Mileage': 170000, 'URL': '/marketplace/item/3549035535373825/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD'}, {'City': 'Riverview', 'Province': 'NB', 'Year': 2019, 'Make': 'Honda', 'Model': 'civic', 'Price': 24999, 'Mileage': 61000, 'URL': '/marketplace/item/255560547337016/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD'}, {'City': 'Moncton', 'Province': 'NB', 'Year': 2018, 'Make': 'Honda', 'Model': 'civic', 'Price': 20495, 'Mileage': 149000, 'URL': '/marketplace/item/263120729766969/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD'}, {'City': 'Charlottetown', 'Province': 'PE', 'Year': 2010, 'Make': 'Honda', 'Model': 'civic', 'Price': 6500, 'Mileage': 159000, 'URL': '/marketplace/item/249495091262857/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD'}, {'City': 'Halifax', 'Province': 'NS', 'Year': 201

In [8]:
# Convert to DataFrame
vehicle_df = pd.DataFrame(vehicles_list)
vehicle_df.tail()

,City,Province,Year,Make,Model,Price,Mileage,URL
545,Vancouver,BC,2016,Honda,civic,23250,74000,/marketplace/item/1051329789364537/?ref=search...
546,Surrey,BC,2017,Honda,civic,23299,61000,/marketplace/item/280979407867602/?ref=search&...
547,Surrey,BC,2007,Honda,civic,5900,103000,/marketplace/item/822504366053233/?ref=search&...
548,New Westminster,BC,2014,Honda,civic,14990,93000,/marketplace/item/941618010468070/?ref=search&...
549,Richmond,BC,2019,Honda,civic,22999,85000,/marketplace/item/254394777375497/?ref=search&...


In [9]:
# Drop duplicates based on URL
vehicle_df = vehicle_df.drop_duplicates(subset='URL')
vehicle_df.info()
vehicle_df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 434 entries, 0 to 549
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   City      434 non-null    object
 1   Province  434 non-null    object
 2   Year      434 non-null    int64 
 3   Make      434 non-null    object
 4   Model     434 non-null    object
 5   Price     434 non-null    int64 
 6   Mileage   434 non-null    int64 
 7   URL       434 non-null    object
dtypes: int64(3), object(5)
memory usage: 30.5+ KB


,City,Province,Year,Make,Model,Price,Mileage,URL
0,Dieppe,NB,2015,Honda,civic,14000,170000,/marketplace/item/3549035535373825/?ref=search...
1,Riverview,NB,2019,Honda,civic,24999,61000,/marketplace/item/255560547337016/?ref=search&...
2,Moncton,NB,2018,Honda,civic,20495,149000,/marketplace/item/263120729766969/?ref=search&...
3,Charlottetown,PE,2010,Honda,civic,6500,159000,/marketplace/item/249495091262857/?ref=search&...
4,Halifax,NS,2011,Honda,civic,9000,125000,/marketplace/item/183227231277501/?ref=search&...


In [10]:
# Drop entries with where Mileage = 0
vehicle_df = vehicle_df[vehicle_df['Mileage'] != 0]

# Print information about the cleaned DataFrame
vehicle_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 434 entries, 0 to 549
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   City      434 non-null    object
 1   Province  434 non-null    object
 2   Year      434 non-null    int64 
 3   Make      434 non-null    object
 4   Model     434 non-null    object
 5   Price     434 non-null    int64 
 6   Mileage   434 non-null    int64 
 7   URL       434 non-null    object
dtypes: int64(3), object(5)
memory usage: 30.5+ KB


In [11]:
vehicle_df.head()

,City,Province,Year,Make,Model,Price,Mileage,URL
0,Dieppe,NB,2015,Honda,civic,14000,170000,/marketplace/item/3549035535373825/?ref=search...
1,Riverview,NB,2019,Honda,civic,24999,61000,/marketplace/item/255560547337016/?ref=search&...
2,Moncton,NB,2018,Honda,civic,20495,149000,/marketplace/item/263120729766969/?ref=search&...
3,Charlottetown,PE,2010,Honda,civic,6500,159000,/marketplace/item/249495091262857/?ref=search&...
4,Halifax,NS,2011,Honda,civic,9000,125000,/marketplace/item/183227231277501/?ref=search&...


In [12]:
#Remove vehicles different from the model on the initial query
model = query.split()[1]

vehicle_df_clean = vehicle_df[vehicle_df['Model'].str.lower() == model.lower()]

# Reset the index
vehicle_df_clean = vehicle_df_clean.reset_index(drop=True)

vehicle_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 351 entries, 0 to 350
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   City      351 non-null    object
 1   Province  351 non-null    object
 2   Year      351 non-null    int64 
 3   Make      351 non-null    object
 4   Model     351 non-null    object
 5   Price     351 non-null    int64 
 6   Mileage   351 non-null    int64 
 7   URL       351 non-null    object
dtypes: int64(3), object(5)
memory usage: 22.1+ KB


In [13]:
# Remove special characters and replace with their normal versions
vehicle_df_clean['City'] = vehicle_df_clean['City'].apply(lambda x: unidecode(x))

vehicle_df_clean.head

<bound method NDFrame.head of                 City Province  Year   Make  Model  Price  Mileage   
0             Dieppe       NB  2015  Honda  civic  14000   170000  \
1          Riverview       NB  2019  Honda  civic  24999    61000   
2            Moncton       NB  2018  Honda  civic  20495   149000   
3      Charlottetown       PE  2010  Honda  civic   6500   159000   
4            Halifax       NS  2011  Honda  civic   9000   125000   
..               ...      ...   ...    ...    ...    ...      ...   
346        Vancouver       BC  2016  Honda  civic  23250    74000   
347           Surrey       BC  2017  Honda  civic  23299    61000   
348           Surrey       BC  2007  Honda  civic   5900   103000   
349  New Westminster       BC  2014  Honda  civic  14990    93000   
350         Richmond       BC  2019  Honda  civic  22999    85000   

                                                   URL  
0    /marketplace/item/3549035535373825/?ref=search...  
1    /marketplace/item/2555

In [14]:
# Convert 'Make' and 'Model' columns to title case
vehicle_df_clean['Make'] = vehicle_df_clean['Make'].str.title()
vehicle_df_clean['Model'] = vehicle_df_clean['Model'].str.title()

# Display the updated DataFrame
vehicle_df_clean.tail()

,City,Province,Year,Make,Model,Price,Mileage,URL
346,Vancouver,BC,2016,Honda,Civic,23250,74000,/marketplace/item/1051329789364537/?ref=search...
347,Surrey,BC,2017,Honda,Civic,23299,61000,/marketplace/item/280979407867602/?ref=search&...
348,Surrey,BC,2007,Honda,Civic,5900,103000,/marketplace/item/822504366053233/?ref=search&...
349,New Westminster,BC,2014,Honda,Civic,14990,93000,/marketplace/item/941618010468070/?ref=search&...
350,Richmond,BC,2019,Honda,Civic,22999,85000,/marketplace/item/254394777375497/?ref=search&...


In [15]:
# Add "facebook.com" to the URLs
vehicle_df_clean['URL'] = 'facebook.com' + vehicle_df_clean['URL']
vehicle_df_clean.sample(20)

,City,Province,Year,Make,Model,Price,Mileage,URL
242,Calgary,AB,2015,Honda,Civic,13800,74000,facebook.com/marketplace/item/843360257305514/...
107,Caledon,ON,2013,Honda,Civic,10500,158000,facebook.com/marketplace/item/247747438202183/...
104,Toronto,ON,2007,Honda,Civic,4999,175000,facebook.com/marketplace/item/983227116135872/...
241,Calgary,AB,2006,Honda,Civic,6000,186000,facebook.com/marketplace/item/6722131351151588...
97,St-Colomban,QC,2005,Honda,Civic,1000,174000,facebook.com/marketplace/item/1964954270533365...
310,Coquitlam,BC,2014,Honda,Civic,14956,168000,facebook.com/marketplace/item/1922321464821123...
20,Warwick,QC,2018,Honda,Civic,20499,88000,facebook.com/marketplace/item/3505153003042727...
72,Montreal,QC,2006,Honda,Civic,5000,200000,facebook.com/marketplace/item/6209663782494489...
148,Hamilton,ON,2017,Honda,Civic,17500,154000,facebook.com/marketplace/item/3444876205774161...
205,Winnipeg,MB,2016,Honda,Civic,21990,151000,facebook.com/marketplace/item/1072840747013590...


In [17]:
#Add vehicle descriptions to the dataframe

# Start the Chrome WebDriver
browser = webdriver.Chrome()

for index, row in vehicle_df_clean.iterrows():
    url = row['URL']
    full_url = f"https://{url}"
    browser.get(full_url)

    # Wait for the "See more" button to be visible and clickable
    try:
        wait = WebDriverWait(browser, 5)
        seemore_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//span[text()="See more"]')))
        seemore_button.click()
        
    except (TimeoutException, NoSuchElementException, ElementNotInteractableException) as e:
        pass

    try: 
        # Find the <div> element containing the specified text
        target_span_element = wait.until(EC.presence_of_element_located((By.XPATH, '//div[@class="xz9dl7a x4uap5 xsag5q8 xkhd6sd x126k92a"]')))

        # Get the text of the target <div> element
        extracted_text = target_span_element.text

        # Remove emojis from the text
        cleaned_text = emoji.demojize(extracted_text)

        # Replace special characters with their regular versions
        cleaned_text = unidecode(cleaned_text)

        # Remove the "[hidden information]" text
        cleaned_text = cleaned_text.replace("[hidden information]", "")

        # Remove the "See less" text
        cleaned_text = cleaned_text.replace("See less", "")

        # Remove remaining emogis
        cleaned_text = re.sub(r":(.*?):", "", cleaned_text)

        # Replace newline characters with a single space
        cleaned_text = cleaned_text.replace("\n", " ")

        # Detect the language of the text
        detected_lang = detect(cleaned_text)

        # Create a translator object
        translator = Translator()


        # Translate the text from French to English
        if detected_lang == 'fr':
            # Translate the text from French to English
            cleaned_text = translator.translate(cleaned_text, src='fr', dest='en').text

        # Update the "Description" column in the DataFrame
        vehicle_df_clean.at[index, "Description"] = cleaned_text
        
        print(f'URL:{full_url}')
        print("")
        print(f"Extracted Text ({index})")
        print(cleaned_text)
        print("-----------------------------")
        
    except (TimeoutException, NoSuchElementException, ElementNotInteractableException) as e:
        print(f'URL:{full_url}')
        print("")
        print(f"Extracted Text ({index})")
        print("No text to extract")
        print("-----------------------------")
        
        pass




URL:https://facebook.com/marketplace/item/3549035535373825/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD

Extracted Text (0)
***PREFERABLY TRADES***(would want something standard)  RTX rims  New tires Stock engine  Godspeed coilovers Hood spacers Sun strip Everything's in great condition(like new) 
-----------------------------
URL:https://facebook.com/marketplace/item/255560547337016/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD

Extracted Text (1)
2019 Honda Civic EX Great fuel efficient car, drives great and has recently new tires.  Comes with a newly installed hitch.  Reason for selling is we want to get an SUV.  No Trades. 
-----------------------------
URL:https://facebook.com/marketplace/item/263120729766969/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD

Extracted Text (2)
LOADED CIVIC EX! One year warranty at no cost, all applicants accepted! Apply below    http://zakthecarrobot.com/apply-today  2018 Honda Civ

URL:https://facebook.com/marketplace/item/1458539804685229/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD

Extracted Text (18)
Attention all car enthusiasts and style-seekers! We have an amazing offer that will elevate your driving experience to new heights. The 2016 Honda Civic Touring is now available for sale at an unbeatable price of just $22,921, plus applicable taxes and fees!  Note: Prices and availability are subject to change. Taxes and fees may vary based on your location. Contact us for more details and purchase options. Please call  or send me a messege! 
-----------------------------
URL:https://facebook.com/marketplace/item/1545149072689077/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD

Extracted Text (19)
2019 Honda Civic Sport Hatchback New Condition! No issues! 65,000kms  Financing available on approved credit!  Fully loaded, heated seats, Honda sensing, adaptive cruise control, lane keep assist, apple car play, floor mats, 

URL:https://facebook.com/marketplace/item/1475916109821523/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD

Extracted Text (31)
2010 Honda Civic Sdn DX-G | 1.8L 4Cyl | FWD | Rebuilt Title  197.000KM  1.8L 4Cyl Front Wheel Drive Cruise Control AC AM/ FM/ CD AS IS  Can be seen and test driven at 103 Cobequid Road Lower Sackville.   We are open: * Mon-Fri 9am - 5pm * Sat 9am - 2pm  9O2-252-4422 and press 1 (Lower Sackville)  #capitalauto #capital auto 
-----------------------------
URL:https://facebook.com/marketplace/item/272056468777263/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD

Extracted Text (32)
Honda Civic 2017 EX FWD *Camera de Recul * Toit Ouvrant * (NIV  23085B - VID: 9826139) chez Olivier Honda Bathurst. Visitez notre site web pour plus d'information.  -Deflecteur de Capot et de Fenetre  - Phares Antibrouillard   -Honda Sensing   -Double Climatisation  Why choose Olivier Honda Bathurst ?   Olivier Honda Bathurst has your Car, Truck

URL:https://facebook.com/marketplace/item/269392029047088/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD

Extracted Text (42)
HONDA CIVIC 2019 LX EDITION / INTERIEUR EN TISSU  CHAUFFANT / BLUETOOTH / CAMERA DE RECUL / ROUES 16 POUCES / CRUISE CONTROL ADAPTATIF + ANTI-COLLISION ET CHANGEMENT DE VOIE / AVEC SEULEMENT 64500 KM CERTIFIE ( FEMME UNIQUE PROPRIETAIRE,RAPPORT CARFAX DISPONIBLE) MOTEUR 2.0 LITRES OFFRANT L'EXCELLENTE PUISSANCE DE 158 HP ET 138 LB-FT TOUT EN ETANT DURABLE ET TRES ECONOMIQUE. TRANSMISSION AUTOMATIQUE AVEC LE MODE SHIFTRONIC,CAMERA DE RECUL,INTERIEUR EN TISSU CHAUFFANT,AIR CLIMATISE,GROUPE ELECTRIQUE COMPLET,LECTEUR CD/MP3 A ECRAN TACTILE COMPATIBLE APPLECARPLAY ET ANDROID AUTO, BLUETOOTH, CRUISE CONTROL VOLANT AJUSTABLE ET TELESCOPIQUE, RPM, FREIN ABS, TELE-VERROUILLAGE, PHARES AUTOMATIQUE.GARANTIE DE 3 MOIS OU 5000 KM MOTOPROPULSEUR INCLUS ( GARANTIE PROLONGE DISPONIBLE A PRIX IMBATTABLE AVEC GARANTIE NATIONAL ) FINANCEMENT RAPIDE ET AVANTA

URL:https://facebook.com/marketplace/item/196555430058686/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD

Extracted Text (53)
Honda Civic Sedan 2016 4DR CVT (NIV 18381B - VID: 9701745) in Levis Subaru.Visit our website for more information.Never accident ||Very nice Civic Touring.Any team with Leather GPS Sunroof Power Group Bluetooth A/C Back Up Camera Cruise Control and Much Much More.1 year warranty 20000km Motoprop group over 75 used vehicles Subaru in inventory!---- In total more than 150 cars in inventory, Toyota, Mazda, Kia, Honda, Hyundai, Jeep, Nissan, Volkswagen, BMW, Chevrolet, Audi, Land Rover, Porsche, Mercedes, Ram, Mini Cut, Ford inspect in152 points for an optimal condition worthy of a real subaru dealer.Why buy any other model when you can enjoy the reliability and the incredible integral traction of a Subaru.Www.levissubaru.com - We are the biggest used vehicle dealer Subaru in Quebec!Our high quality standards, the largest choice in the country,

...

URL:https://facebook.com/marketplace/item/825887938937377/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD

Extracted Text (289)
Old vehicle but good 
-----------------------------
URL:https://facebook.com/marketplace/item/613084860960731/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD

Extracted Text (290)
Clean title Runs strong and smooth Ac cold very good rubber Brakes are 90 % All power options lady driven no smoke no pets daily driving car Used engine replaced in October last year was low compression in one cylinder on previous engine so put new one with only 103xxx kms on it odometer shows 298xxx kms lots of life left on this car Price to sell 
-----------------------------
URL:https://facebook.com/marketplace/item/3243180352646834/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD

Extracted Text (291)
Honda Civic 2015 EX Silver All services & Oil change have been done on time. CLEAN TITLE.  In excellent condition and c

URL:https://facebook.com/marketplace/item/1010123390442882/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD

Extracted Text (303)
Great excellent car in good shape low mileage runs great passenger side mirror needs replacement which cost $150 and squilling coming from the fan belt just needs a new belt other then that runs great new tires none rebuilt no accidents 
-----------------------------
URL:https://facebook.com/marketplace/item/1240112890040086/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD

Extracted Text (304)
2016 Honda Civic  Run drive very well   Fully detail   Automatic transmission  Automatic window  Back up camera  Leather seat   Bluetooth  Heated seat   Ac works great   Cruise control    Rebuild title     Ask for more information  
-----------------------------
URL:https://facebook.com/marketplace/item/692435789390846/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD

Extracted Text (305)
Honda Civic EX 2016 

URL:https://facebook.com/marketplace/item/309874814722670/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD

Extracted Text (319)
Excellent condition  LOW MILEAGE ,  Back up camera ,  Bluetooth, Heated seats ,  Neat & clean  Almost New tyres ,  Eco mode , rebuilt title  
-----------------------------
URL:https://facebook.com/marketplace/item/6160161327443013/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD

Extracted Text (320)
2007 Honda Civic title is a rebuilt I fix the car myself. I have a body shop engine size 1.7 the car in a really good condition. Does not have any problem tomorrow information you can come with the sunroof back up sensor as well message me. 
-----------------------------
URL:https://facebook.com/marketplace/item/1262471611081844/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD

Extracted Text (321)
2012 Honda Civic Coupe EX 1.8L 4 Cylinder engine - Fuel Efficient! Very well maintained - Service history s

URL:https://facebook.com/marketplace/item/812923636908675/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD

Extracted Text (333)
The 2019 Honda Civic LX comes jam packed with a ton of standard features including a digital cluster, large center display with Apple CarPlay and Android Auto, heated seats, backup camera, and much more. This Civic also comes equipped with Honda's suite of standard safety features such as Forward Collision Warning, Lane Departure Warning, and Adaptive Cruise Control! 
-----------------------------
URL:https://facebook.com/marketplace/item/636430331776263/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD

Extracted Text (334)
YES IT IS AVAILABLE  NO OFFERS.  FIRM + DOC FEE + TAX THANK YOU.  REBUILT DUE TO A VERY MINOR DAMAGE, ALL PIX AVAILABLE.   2020 Honda Civic Touring! Powerful and yet super fuel efficient 1.5 Litre Turbo Charged Engine Combined with the 8 Speed Automatic Transmission 5 Passenger.  Super Cheap on Gas  

URL:https://facebook.com/marketplace/item/1036189614030854/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD

Extracted Text (345)
- Local BC and Civic Touring! which comes with an extended 7-year/160,000kms limited warranty   "CLEAN TITLE"  - Touring top trim level with Sunroof and Navigation, fully loaded - contact me NOW before it's gone! - Comes with Honda Sensing Safety technologies:  * Collision Mitigation Braking System (CMBS) * Forward Collision Warning (FCW) * Lane Departure Warning (LDW) * Road Departure Mitigation (RDM) * Adaptive Cruise Control (ACC) * Lane Keeping Assist System (LKAS) - Practical and fuel efficient with a 1.5L turbocharged engine, and versatile 60/40 split-folding rear seats. / Heated Rear Seats // Proximity key entry system with Push-Button Start / Remote Engine Start / Apple CarPlay and Android Auto // multi-angle Rearview camera with dynamic guidelines // Bluetooth // Heated and power-adjustable front seats // dual zone automatic A/C 

In [18]:
# Close the browser
browser.quit()

In [20]:
vehicle_df_clean.tail(20)

,City,Province,Year,Make,Model,Price,Mileage,URL,Description
331,Surrey,BC,2018,Honda,Civic,22500,60000,facebook.com/marketplace/item/1423749668357859...,2018 Honda Civic Sedan Touring CVT Driven onl...
332,Vancouver,BC,2016,Honda,Civic,23999,163000,facebook.com/marketplace/item/1231311474241755...,Accident-free 2016 Honda Civic LX Carfax avail...
333,Richmond,BC,2019,Honda,Civic,22999,85000,facebook.com/marketplace/item/812923636908675/...,The 2019 Honda Civic LX comes jam packed with ...
334,New Westminster,BC,2020,Honda,Civic,22999,112000,facebook.com/marketplace/item/636430331776263/...,YES IT IS AVAILABLE NO OFFERS. FIRM + DOC FE...
335,New Westminster,BC,2012,Honda,Civic,13998,154000,facebook.com/marketplace/item/1198780040790442...,Pre-Owned 2012 Honda Civic EX-L Sedan CALL/TE...
336,Vancouver,BC,2019,Honda,Civic,25995,85000,facebook.com/marketplace/item/3632215833728421...,CALL LP (778) (548) (4988) 2019 Honda Civic S...
337,Surrey,BC,2015,Honda,Civic,13500,175000,facebook.com/marketplace/item/311780391293484/...,Selling Honda Civic 2015 rebuilt. Fully loade...
338,Chilliwack,BC,2017,Honda,Civic,23998,106000,facebook.com/marketplace/item/783980433460647/...,"2017 Civic Hatchback Sport Touring! 106,186 Km..."
339,Richmond,BC,2015,Honda,Civic,19500,85000,facebook.com/marketplace/item/804696244445412/...,Clean title. Zero accident finance available ...
340,Richmond,BC,2001,Honda,Civic,7995,58000,facebook.com/marketplace/item/1248621249179510...,"ONLY 58XXXKM This is a Local, No Accident, On..."


In [87]:
# Convert the 'Description' column to strings and replace NaN values with empty strings
vehicle_df_clean['Description'] = vehicle_df_clean['Description'].astype(str).fillna('')

# Join all the text elements in the 'Description' column into a single string with space as the separator
text_combined = ' '.join(vehicle_df_clean['Description'])

# Convert the string to lowercase and split it into individual words
words_list = text_combined.lower().split()

# Remove words with two letters or less
words_list = [word for word in words_list if len(word) > 2]

# Create a pandas Series with the list of words and calculate word frequency counts
word_counts = pd.Series(words_list).value_counts()

# Words to be excluded from the final series
words_to_exclude = ['and', 'the', 'with', 'honda', 'civic', 'for', 'car', 'your', 'new', 
                    'our', 'you', 'vehicle', 'all', 'has', 'power', 'financing']

# Exclude the specified words from the word_counts series
filtered_word_counts = word_counts.drop(words_to_exclude, errors='ignore')

# Get the top 100 most frequently occurring words
top_100_words = filtered_word_counts[:100]

top_100_words

clean      97
this       97
very       86
have       82
tires      82
           ..
fuel       26
website    26
windows    26
side       26
carfax     26
Name: count, Length: 100, dtype: int64

In [41]:
# Step 2: Tokenization (split text into sentences)
car_descriptions = ' '.join(vehicle_df_clean['Description'].tolist())  # Combine all descriptions into one string
sentences = sent_tokenize(car_descriptions)

# Step 3: Stopword Removal and Counting
stop_words = set(stopwords.words('english'))
filtered_sentences = [sentence.lower() for sentence in sentences if sentence.lower() not in stop_words]
statement_counts = Counter(filtered_sentences)

# Step 4: Sort and Select the most common statements
most_common_statements = statement_counts.most_common(20)[10:20]# Select the top 10 most common statements

# Display the most common statements
for statement, count in most_common_statements:
    print(f"Statement: {statement}")
    print(f"Frequency Count: {count}\n")

Statement: contact us today or visit our website to explore our inventory, take advantage of our exceptional financing options, and enjoy the added perks that make car buying a breeze.
Frequency Count: 14

Statement: financing available trade ins available  all types of credit accepted  for more info - call/message 204- 380- 1785-  book your test drive today !
Frequency Count: 4

Statement: visit our website for more information.
Frequency Count: 4

Statement: inventory published with the help of marketplace boost by d2c media.
Frequency Count: 4

Statement: visitez notre site web pour plus d'information.
Frequency Count: 3

Statement: olivier honda bathurst has your car, truck, suv or little sports car that you are looking for.
Frequency Count: 3

Statement: we have a little of everything from honda, ford, toyota, hyundai, acura, kia, mazda, chrysler, dodge, jeep, ram, chevrolet, gmc, bmw, lexus, cadillac, buick, nissan, audi, volkswagen, subaru, tesla, mercedes.
Frequency Count: 3

S

In [80]:
# Tokenize the sentences using word_tokenize and then check for each word in the sentence if it matches any of 
# The target terms. If a match is found, we collect the 4 words before and after the matched word to form the matching_sentence.
# Only these sentences with a match and limited context are stored in the matching_sentences_dict dictionary.

#Create an empty list to store the matching results
matching_results_list = []

#Compile a regex pattern to match any of the target terms
target_terms = ['rebuilt', 'accident', 'clean', 'carfax', 'damage', 'car fax', 'car-fax', 'recertified']
pattern = re.compile(r'\b(?:' + '|'.join(target_terms) + r')\b', flags=re.IGNORECASE)

#Iterate through the DataFrame and find rows with the specified terms in the description
for index, row in vehicle_df_clean.iterrows():
    description = row['Description'].lower()
    url = row['URL']
    
    if pattern.search(description):
        sentences = sent_tokenize(description)
        matching_sentences = []
        for sentence in sentences:
            words = word_tokenize(sentence)
            for i, word in enumerate(words):
                if pattern.search(word):
                    start_index = max(0, i - 4)
                    end_index = min(i + 5, len(words))
                    matching_sentence = ' '.join(words[start_index:end_index])
                    matching_sentences.append(matching_sentence)
        if matching_sentences:
            matching_result = {'Index': index, 'Sentences': matching_sentences, 'URL': url}
            matching_results_list.append(matching_result)

#Create a DataFrame from the list of dictionaries
result_df = pd.DataFrame(matching_results_list)

#Save the DataFrame as a CSV file
result_df.to_csv('matching_results.csv', index=False)

In [81]:
len(matching_urls_dict)

168

In [89]:
# Create CSV file
vehicle_df_clean.to_csv('vehicles_clean_description.csv', header=True, index=False)

In [86]:
#Read the CSV file as a Pandas DataFrame
df_filtered_matching_results = pd.read_csv('matching_results_filtered.csv')
df_filtered_matching_results.head()

,Index,Description,URL,Rebuilt
0,11,['driven ( approximately ) clean title comes w...,facebook.com/marketplace/item/306105255150907/...,0
1,19,"['clean car fax , fresh']",facebook.com/marketplace/item/1545149072689077...,0
2,21,"['mag , bluetooth , carfax without accident.19...",facebook.com/marketplace/item/330387509316880/...,0
3,27,['but has never been accident.the interviews h...,facebook.com/marketplace/item/821401316043008/...,0
4,31,['4cyl | fwd | rebuilt title 197.000km 1.8l 4c...,facebook.com/marketplace/item/1475916109821523...,1
